In [1]:
import pandas as pd

In [2]:
path = 'Data/raw_data/'
file_name = 'Transaction.csv' 

In [4]:
df = pd.read_csv(path+file_name)
df.head(2)

,Transaction ID,Customer ID,Product ID,Product Category,Purchase Amount,Purchase Date
0,1,238,45,Sensitive Skin Treatment,25,2024-03-24
1,1,238,35,Lip Care,15,2024-03-24


In [5]:
# Convert 'Purchase Date' to datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

In [11]:
product_file = 'Product.csv'
product_df = pd.read_csv(path+product_file)
product_df.head(2)

,Product ID,Product Category,SKU,Product Price ($)
0,1,Cleansers,Gel Cleansers,20
1,2,Cleansers,Foam Cleanser,15


In [12]:
# Merge transaction data with product data to get SKU info
merged_df = df.merge(product_df[['Product ID', 'SKU']], on='Product ID', how='left')
merged_df.head(2)

,Transaction ID,Customer ID,Product ID,Product Category,Purchase Amount,Purchase Date,SKU
0,1,238,45,Sensitive Skin Treatment,25,2024-03-24,Calming Serums
1,1,238,35,Lip Care,15,2024-03-24,Lip Scrub


### Top Selling Products & Catgories

In [13]:
# Identify top-10 selling products
top_product_ids = top_products.index.tolist()
top_selling_with_sku = product_df[product_df['Product ID'].isin(top_product_ids)][['Product ID', 'SKU']]

top_selling_with_sku = top_selling_with_sku.set_index('Product ID').loc[top_product_ids].reset_index()
top_selling_with_sku['Sales Count'] = top_products.values

In [19]:
top_selling_with_sku.head(10)

,Product ID,SKU,Sales Count
0,45,Calming Serums,1132
1,44,Soothing Creams,1081
2,35,Lip Scrub,928
3,4,Micellar Water,918
4,19,Cream Moisturizer,909
5,18,Gel Moisturizer,898
6,1,Gel Cleansers,898
7,21,Water-Based Moisturizer,894
8,22,Night Cream,886
9,36,Lip Mask,882


In [23]:
# Identify top-3 selling categories
top_category_counts = df['Product Category'].value_counts().head(10).reset_index()
top_category_counts.columns = ['Product Category', 'Count']

In [25]:
top_category_counts.head(3)

,Product Category,Count
0,Moisturizers,4460
1,Cleansers,4427
2,Lip Care,2686


### Average Spending per Customer per Transaction

In [28]:
# Group by Transaction ID to get total spending per transaction
transaction_spending = df.groupby('Transaction ID').agg({
    'Customer ID': 'first',  # assuming one customer per transaction
    'Purchase Amount': 'sum'
}).reset_index()

# Group by Customer ID to calculate average transaction spending per customer
customer_avg_spending = transaction_spending.groupby('Customer ID')['Purchase Amount'].agg(
    Total_Spending='sum',
    Number_of_Transactions='count',
    Average_Spending_Per_Transaction='mean'
).reset_index()

customer_avg_spending['Average_Spending_Per_Transaction'] = customer_avg_spending['Average_Spending_Per_Transaction'].round(2)

In [29]:
customer_avg_spending.head(10)

,Customer ID,Total_Spending,Number_of_Transactions,Average_Spending_Per_Transaction
0,1,1010,12,84.17
1,2,1370,14,97.86
2,3,815,10,81.50
3,4,465,6,77.50
4,5,760,9,84.44
5,6,705,11,64.09
6,7,670,9,74.44
7,8,790,9,87.78
8,9,640,8,80.00
9,10,1190,14,85.00


### Average Spending per Transaction

In [30]:
# Calculate total and average spending per transaction
transaction_summary = df.groupby('Transaction ID').agg(
    Customer_ID=('Customer ID', 'first'),
    Transaction_Spending=('Purchase Amount', 'sum')
).reset_index()

average_transaction_spending = transaction_summary['Transaction_Spending'].mean()
average_transaction_spending = round(average_transaction_spending, 2)

print(average_transaction_spending)

90.73


In [33]:
# Find lowest and highest spending in a transaction
min_transaction_spending = transaction_summary['Transaction_Spending'].min()
max_transaction_spending = transaction_summary['Transaction_Spending'].max()

print(min_transaction_spending, max_transaction_spending)

20 200
